<a href="https://colab.research.google.com/github/sraeisi/MachineLearning_Physics/blob/master/Lec_7/Convnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

import requests
import io

## Farsi Hand-written digits classification

In [0]:
'''
Let's import the Farsi digits (your assignment)
'''

url = 'https://raw.githubusercontent.com/sraeisi/MachineLearning_Physics/master/Assignments/A3/X.npz'
s=requests.get(url, stream=True).content
X = np.load(io.BytesIO(s) )['arr_0']
url = 'https://raw.githubusercontent.com/sraeisi/MachineLearning_Physics/master/Assignments/A3/Y.npz'
s=requests.get(url, stream=True).content
Y = np.load(io.BytesIO(s) )['arr_0']

from sklearn.utils import shuffle
X, Y = shuffle(X,Y, random_state = 0 )

In [0]:
print('Each image is of the form: {}'.format(X[0].shape ) )

Each image is of the form: (32, 32)


In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 32)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Flatten(input_shape=(n_w , n_h) ))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               131200    
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total params: 132,490
Trainable params: 132,490
Non-trainable params: 0
_________________________________________________________________


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,Y_train, 
                    epochs=5, batch_size = 64, 
                   validation_data = (X_test, Y_test))

Train on 76764 samples, validate on 25588 samples
Epoch 1/5
76764/76764 [==============================] - 4s 55us/sample - loss: 0.0328 - acc: 0.9903 - val_loss: 0.0532 - val_acc: 0.9844
Epoch 2/5
76764/76764 [==============================] - 4s 55us/sample - loss: 0.0224 - acc: 0.9930 - val_loss: 0.0527 - val_acc: 0.9848
Epoch 3/5
76764/76764 [==============================] - 4s 48us/sample - loss: 0.0169 - acc: 0.9949 - val_loss: 0.0501 - val_acc: 0.9855
Epoch 4/5
76764/76764 [==============================] - 4s 48us/sample - loss: 0.0118 - acc: 0.9961 - val_loss: 0.0564 - val_acc: 0.9869
Epoch 5/5
76764/76764 [==============================] - 4s 48us/sample - loss: 0.0091 - acc: 0.9973 - val_loss: 0.0520 - val_acc: 0.9870


In [0]:
Y_predicted = np.argmax( model.predict(X_test) , axis=1 )
cm = confusion_matrix(Y_test, Y_predicted)
print(cm)

[[2518    0    0    0    2   14    0    1    1    0]
 [   1 2501    3    0    1    0    3    0    0    2]
 [   2    8 2458   16    7    0    2    6    0    1]
 [   0    0   52 2452   34    3    0    3    0    0]
 [   1    4   12   23 2629   11    5    3    1    3]
 [   4    1    1    1    0 2483    4    0    1    2]
 [   0    4    2    1    2    1 2570    8    2   18]
 [   2    1    3    1    3    0    0 2598    0    0]
 [   0    5    0    0    1    1    2    0 2523   16]
 [   0    6    2    0    2    0    8    0    3 2523]]


## Convnet

In [0]:
X_train[0].shape

(32, 32)

In [0]:
n_w , n_h = X[0].shape
from keras import layers
from keras import models

model = models.Sequential()

model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=(n_w , n_h, 1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 30, 30, 16)        160       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 4, 4, 64)          18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 256)               0         
__________

In [0]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

X_train = X_train.reshape(-1, n_w, n_h, 1)
X_test = X_test.reshape(-1, n_w, n_h, 1)

history = model.fit(X_train,Y_train, 
                    epochs=5, batch_size = 64, 
                   validation_data = (X_test, Y_test))

Train on 76764 samples, validate on 25588 samples
Epoch 1/5
76764/76764 [==============================] - 7s 94us/step - loss: 0.1433 - acc: 0.9566 - val_loss: 0.0391 - val_acc: 0.9881
Epoch 2/5
76764/76764 [==============================] - 6s 74us/step - loss: 0.0331 - acc: 0.9899 - val_loss: 0.0333 - val_acc: 0.9898
Epoch 3/5
76764/76764 [==============================] - 6s 72us/step - loss: 0.0236 - acc: 0.9925 - val_loss: 0.0257 - val_acc: 0.9912
Epoch 4/5
76764/76764 [==============================] - 6s 72us/step - loss: 0.0171 - acc: 0.9944 - val_loss: 0.0230 - val_acc: 0.9932
Epoch 5/5
76764/76764 [==============================] - 6s 72us/step - loss: 0.0149 - acc: 0.9951 - val_loss: 0.0212 - val_acc: 0.9935


In [0]:
Y_predicted = np.argmax( model.predict(X_test ), axis=1 )
cm = confusion_matrix(Y_test, Y_predicted)
print(cm)

[[2532    0    0    0    0    3    0    0    0    1]
 [   3 2502    3    0    0    0    0    0    1    2]
 [   2    0 2489    9    0    0    0    0    0    0]
 [   0    0   25 2512    5    0    2    0    0    0]
 [   0    0    8   21 2657    0    6    0    0    0]
 [   6    2    0    0    2 2470    4    0    5    8]
 [   0    0    5    0    5    2 2586    0    0   10]
 [   3    2    8    0    1    0    0 2594    0    0]
 [   0    2    0    0    1    1    0    0 2544    0]
 [   0    3    0    0    0    0    5    0    0 2536]]


In [0]:
'''Implement LeNet-5'''